In [194]:
from dotenv import load_dotenv
from metagame import TradingClient
import os

load_dotenv()

client = TradingClient(
    api_url=os.environ["API_URL"],
    jwt=os.environ["JWT"],
    act_as=0,
)

In [195]:
import pandas as pd

def get_market_trades(market_name: str):
    state = client.state()
    market_id = state.market_name_to_id.get(market_name)    
    if market_id is None:
        return pd.DataFrame()

    if not state.markets[market_id].hasFullTradeHistory:
        client.get_full_trade_history(market_id)
        state = client.state()

    trades = state.markets[market_id].trades
    if not trades:
        return pd.DataFrame()
    df = pd.DataFrame(trades).set_index("id")
    df["buyer"] = df["buyer_id"].map(lambda x: state.accounts[x].name)
    df["seller"] = df["seller_id"].map(lambda x: state.accounts[x].name)
    df["timestamp"] = df["transaction_id"].map(lambda x: state.transactions[x])
    return df.drop(columns=["market_id", "buyer_id", "seller_id", "transaction_id"])


In [202]:
def get_market_positions(market_name: str):
    df = get_market_trades(market_name)
    if df.empty:
        return pd.DataFrame()
    return pd.concat([
        df.groupby("buyer")["size"].sum(),
        -df.groupby("seller")["size"].sum()
    ]).groupby(level=0).sum()

def get_multiple_market_positions(market_names: list[str]):
    return pd.DataFrame({market: get_market_positions(market) for market in market_names}).fillna(0)

def get_market_avg_fills(market_name: str):
    df = get_market_trades(market_name)
    if df.empty:
        return pd.DataFrame()
    df["fill_size"] = df["size"] * df["price"]
    contracts_bought = df.groupby("buyer")["size"].sum()
    contracts_sold = df.groupby("seller")["size"].sum()
    buy_fills = df.groupby("buyer")["fill_size"].sum()
    sell_fills = df.groupby("seller")["fill_size"].sum()
    avg_buy_fill = buy_fills / contracts_bought
    avg_sell_fill = sell_fills / contracts_sold
    last_price = df.iloc[-1]["price"]
    df = pd.DataFrame({
        "contracts_bought": contracts_bought,
        "contracts_sold": contracts_sold,
        "avg_buy_fill": avg_buy_fill,
        "avg_sell_fill": avg_sell_fill,
    }).fillna(0)
    df["position"] = df.contracts_bought - df.contracts_sold
    df["mtm"] = df.position * last_price - df.avg_buy_fill * df.contracts_bought + df.avg_sell_fill * df.contracts_sold
    return df


In [203]:
import ipywidgets as widgets

def get_dice(refresh):
    markets = ["min_dice_c", "max_dice_c", "avg_dice_c"]
    dfs = []
    for market in markets:
        df = get_market_avg_fills(market)
        df.index = pd.MultiIndex.from_product([[market], df.index])
        dfs.append(df)
    return pd.concat(dfs)

widgets.interact(get_dice, refresh=widgets.Checkbox(description="Refresh Data"))


interactive(children=(Checkbox(value=False, description='Refresh Data'), Output()), _dom_classes=('widget-inte…

<function __main__.get_dice(refresh)>